In [ ]:
from pathlib import Path
from typing import Literal

from shapely.geometry import Point

from s2shores.bathy_debug.spatial_dft_bathy_estimator_debug import SpatialDFTBathyEstimatorDebug
from s2shores.bathy_debug.spatial_correlation_bathy_estimator_debug import SpatialCorrelationBathyEstimatorDebug
from s2shores.waves_exceptions import WavesEstimationError

from utils import initialize_sequential_run, build_dataset

In [ ]:
base_path = Path("../TestsS2Shores").resolve()
test_case: Literal["7_4", "8_2"] = "8_2"
method: Literal["spatial_corr", "spatial_dft", "temporal_corr"] = "spatial_corr"

product_path: Path = base_path / "products" / f"SWASH_{test_case}/testcase_{test_case}.tif"
config_path: Path = base_path / f"reference_results/debug_pointswash_{method}/wave_bathy_inversion_config.yaml"
debug_file: Path = base_path / f"debug_points/debug_points_SWASH_{test_case}.yaml"

estimation_point = Point(451.0, 499.0)

In [ ]:
bathy_estimator, ortho_bathy_estimator, ortho_sequence, config = initialize_sequential_run(
    product_path=product_path,
    config_path=config_path,
    point=estimation_point,
)

If you want to change any parameter of the configuration, modify the values of the object `config` as you would with a dict.  

Example:
```python
config["parameter"] = "new_value"
```

In [ ]:
local_bathy_estimator = SpatialCorrelationBathyEstimatorDebug(
    estimation_point,
    ortho_sequence,
    bathy_estimator,
)

if not local_bathy_estimator.can_estimate_bathy():
    raise WavesEstimationError("Cannot estimate bathy.")

## Preprocess images

In [ ]:
if True:
    local_bathy_estimator.preprocess_images()
else:
    # Write your custom code here.
    ...


## Find direction

In [ ]:
if True:
    estimated_direction = local_bathy_estimator.find_direction()
else:
    # Write your custom code here.
    ...


## Compute radon transforms

In [ ]:
if True:
    local_bathy_estimator.compute_radon_transforms(estimated_direction)
else:
    # Write your custom code here.
    ...


## Compute spatial correlation

In [ ]:
if True:
    correlation_signal = local_bathy_estimator.compute_spatial_correlation(estimated_direction)
else:
    # Write your custom code here.
    ...


## Compute wavelength

In [ ]:
if True:
    wavelength = local_bathy_estimator.compute_wavelength(correlation_signal)
else:
    # Write your custom code here.
    ...


## Compute delta position

In [ ]:
if True:
    delta_position = local_bathy_estimator.compute_delta_position(correlation_signal, wavelength)
else:
    # Write your custom code here.
    ...


## Save wave field estimation

In [ ]:
if True:
    local_bathy_estimator.save_wave_field_estimation(estimated_direction, wavelength, delta_position)
else:
    # Write your custom code here.
    ...


In [ ]:
local_bathy_estimator.bathymetry_estimations[0].is_physical()

In [ ]:
dataset = build_dataset(
    bathy_estimator,
    ortho_bathy_estimator,
    local_bathy_estimator,
)
dataset

## TODO

- Créer une fonction pour les premières étapes ✅
- Documenter les modifications des attributs d'instance de chaque méthode du run
- Eclater la cellule de run ✅
- rentrer dans chaque étape
- flag _DEFAULT pour chaque étape ou custom function ✅
- faire la même chose pour les 3 méthodes



Option: Configuration pydantic